In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
#def scrape():
browser = init_browser()

In [4]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(2)

In [ ]:
news_title = []
news_p = []

In [ ]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
results = soup.find_all("li", class_="slide")
for r in results[0]:
    try: 
        news_title = r.find("div", class_="content_title").get_text()
        news_p = r.find("div", class_="article_teaser_body").get_text()
    except:
        pass
    print(news_title)
    print('')
    print(news_p)

In [ ]:
img_site = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_site)
browser.links.find_by_partial_text('FULL').click()
browser.links.find_by_partial_href('/spaceimages/details').click()
browser.links.find_by_partial_href('/spaceimages/images/largesize/').click()

# https://splinter.readthedocs.io/en/latest/browser.html#verifying-page-url-with-browser-url
featured_image_url = browser.url

# for i in img_url:
#     try:
#         featured_image_url = i.find("figure", class_="lede").find("img")["src"].get_text()
#     except:
#         pass
    

# for img in img_url:
# #     print('page:', x, '-------------')
#     print(img.string)

print(featured_image_url)

In [10]:
hemisphere_image_urls = []


In [11]:
hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hems_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
titles = soup.find_all("div", class_="description")
for t in titles:
    try:
        result = {}
        result["title"] = t.find("h3").get_text()
        hemisphere_image_urls.append(result)
    except:
        pass
    
    print(result)

{'title': 'Cerberus Hemisphere Enhanced'}
{'title': 'Schiaparelli Hemisphere Enhanced'}
{'title': 'Syrtis Major Hemisphere Enhanced'}
{'title': 'Valles Marineris Hemisphere Enhanced'}


In [ ]:
# hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# browser.visit(hems_url)
# browser.links.find_by_partial_text('Cerberus').click()
# cerberus_text = soup.find_all

# title_text = {}

In [ ]:
browser.url

In [ ]:
browser.quit()

In [ ]:
facts_url ='https://space-facts.com/mars/'

mars_facts = pd.read_html(facts_url)

In [ ]:
facts_table = mars_facts[0]
facts_table.head(9)

[{'title': 'Cerberus Hemisphere Enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced'}]